In [1]:
import requests 
import pandas as pd
import webbrowser
from bs4 import BeautifulSoup

### Dados de acesso - Aplicativo (Acess Token)

In [ ]:
Application_client_id = ''
Client_secret_value = ''    
 

### Pegar Acess Token e Refresh Token - Primeira vez

In [ ]:

# Substitua pelos valores corretos do client_id e client_secret
client_id = Application_client_id
client_secret = Client_secret_value
redirect_uri = "https://login.microsoftonline.com/common/oauth2/nativeclient"
scope = "openid profile https://ads.microsoft.com/msads.manage offline_access"

# Abre o navegador para solicitar permissão ao usuário
authorization_url = (
    f"https://login.microsoftonline.com/common/oauth2/v2.0/authorize"
    f"?client_id={client_id}&scope={scope.replace(' ', '%20')}&response_type=code&"
    f"redirect_uri={redirect_uri}&state=12345&prompt=login"
)
webbrowser.open(authorization_url)

# Captura o código de autorização do URI de resposta
code = input("Após conceder permissão, insira o código de autorização aqui: ")

# Verifica se o código foi obtido corretamente
if not code:
    print("Erro: código de autorização não encontrado. Verifique o URI inserido.")
    exit()

# Solicita os tokens de acesso e atualização
token_url = "https://login.microsoftonline.com/common/oauth2/v2.0/token"
token_data = {
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": scope,
    "code": code,
    "grant_type": "authorization_code",
    "redirect_uri": redirect_uri
}

# Requisição para obter o access token e refresh token
response = requests.post(token_url, data=token_data)
oauth_tokens = response.json()

# Verifica se o access token foi obtido com sucesso
if "access_token" not in oauth_tokens:
    print("Erro: token de acesso não obtido.")
    exit()

# Exibe os tokens
print("Access token:", oauth_tokens["access_token"])
print("Access token expires in:", oauth_tokens["expires_in"])
print("Refresh token:", oauth_tokens["refresh_token"])


### Pegar Acess Token pelo Refresh Token

In [ ]:

# Solicita os tokens de acesso e atualização
token_url = "https://login.microsoftonline.com/common/oauth2/v2.0/token"
scope = "openid profile https://ads.microsoft.com/msads.manage offline_access"

Refresh_token = 'client_id = Application_client_id
client_secret = Client_secret_value

# Para renovar o token de acesso usando o refresh token
refresh_data = {
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": scope,
    "grant_type": "refresh_token",
    "refresh_token": Refresh_token
}

# Requisição para renovar o token de acesso
refresh_response = requests.post(token_url, data=refresh_data)
new_oauth_tokens = refresh_response.json()
access_token = new_oauth_tokens['access_token']

# Exibe os novos tokens
print("Novo Access token:", new_oauth_tokens["access_token"])
print("Access token expires in:", new_oauth_tokens["expires_in"])
print("Novo Refresh token:", new_oauth_tokens["refresh_token"])

### Dados de acesso - Requisição

In [ ]:
AuthenticationToken = new_oauth_tokens["access_token"]
Accountid = ''
CustomerAccountId = Accountid
CustomerId = ''
User_id = ''
DeveloperToken = ''
Refresh_token = new_oauth_tokens["refresh_token"]

### Pegando o UserIDs

In [ ]:

# Montar a requisição SOAP
soap_request = f'''<?xml version="1.0" encoding="utf-8"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/">
  <s:Header>
    <h:ApplicationToken i:nil="true" xmlns:h="https://bingads.microsoft.com/Customer/v13" xmlns:i="http://www.w3.org/2001/XMLSchema-instance" />
    <h:AuthenticationToken xmlns:h="https://bingads.microsoft.com/Customer/v13">{access_token}</h:AuthenticationToken>
    <h:DeveloperToken xmlns:h="https://bingads.microsoft.com/Customer/v13">{DeveloperToken}</h:DeveloperToken>
  </s:Header>
  <s:Body>
    <SearchAccountsRequest xmlns="https://bingads.microsoft.com/Customer/v13">
      <Predicates xmlns:a="https://bingads.microsoft.com/Customer/v13/Entities" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
        <a:Predicate>
          <a:Field>UserId</a:Field>
          <a:Operator>Equals</a:Operator>
          <a:Value>{User_id}</a:Value>
        </a:Predicate>
      </Predicates>
      <Ordering i:nil="true" xmlns:a="https://bingads.microsoft.com/Customer/v13/Entities" xmlns:i="http://www.w3.org/2001/XMLSchema-instance" />
      <PageInfo xmlns:a="https://bingads.microsoft.com/Customer/v13/Entities" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
        <a:Index>0</a:Index>
        <a:Size>10</a:Size>
      </PageInfo>
    </SearchAccountsRequest>
  </s:Body>
</s:Envelope>'''

# Cabeçalhos da requisição
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "SearchAccounts"
}

# URL do serviço SOAP
url = "https://clientcenter.api.bingads.microsoft.com/Api/CustomerManagement/v13/CustomerManagementService.svc"

# Enviando a requisição POST
response = requests.post(url, data=soap_request, headers=headers)

# Verificando o status da resposta
if response.status_code == 200:
    print("Requisição bem-sucedida!")
    print("Resposta:")
    print(response.text)  # Aqui você pega a resposta
else:
    print(f"Erro na requisição: {response.status_code}")
    print(response.text)



### Requisição

In [ ]:
# URL do endpoint da API
url = "https://api.bingads.microsoft.com/CampaignManagement/v13"

# Cabeçalhos da requisição
headers = {
    'Content-Type': 'text/xml; charset=utf-8',
    'SOAPAction': 'https://bingads.microsoft.com/CampaignManagement/v13/GetCampaignsByAccountId'
}

# Corpo da requisição SOAP
body = f'''<?xml version="1.0" encoding="utf-8"?>
<s:Envelope xmlns:i="http://www.w3.org/2001/XMLSchema-instance" xmlns:s="http://schemas.xmlsoap.org/soap/envelope/">
  <s:Header xmlns="https://bingads.microsoft.com/CampaignManagement/v13">
    <Action mustUnderstand="1">GetCampaignsByAccountId</Action>
    <ApplicationToken i:nil="false">ValueHere</ApplicationToken>
    <AuthenticationToken i:nil="false">{AuthenticationToken}</AuthenticationToken>
    <CustomerAccountId i:nil="false">{Accountid}</CustomerAccountId>
    <CustomerId i:nil="false">{CustomerId}</CustomerId>
    <DeveloperToken i:nil="false">{DeveloperToken}</DeveloperToken>
  </s:Header>
  <s:Body>
    <GetCampaignsByAccountIdRequest xmlns="https://bingads.microsoft.com/CampaignManagement/v13">
      <AccountId>{Accountid}</AccountId>
      <CampaignType>ValueHere</CampaignType>
    </GetCampaignsByAccountIdRequest>
  </s:Body>
</s:Envelope>'''

# Fazendo a requisição
response = requests.post(url, headers=headers, data=body)

# Exibindo a resposta
print(response.status_code)
print(response.text)